To run this notebook, you will have to use LEAP-Pangeo hub!

<a href="https://leap.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fm2lines%2Fdata-gallery&urlpath=lab%2Ftree%2Fdata-gallery%2Fnotebooks%2FOM4_SE_animation.ipynb&branch=main"><img src="https://custom-icon-badges.demolab.com/badge/LEAP-Launch%20%F0%9F%9A%80-blue?style=for-the-badge&logo=leap-globe" style="height:30px;"></a>

# OM4 Animation

In this notebook, we will visualize the OM4 data using the `xarray` and `matplotlib` libraries and create an animation of the Surface Kinetic Energy over time. We will explore visualizations in both global and regional scales. 

Credits: https://github.com/Pperezhogin/MOM6/blob/Zanna-Bolton-2020/experiments/ZB20-Results/14-Global-animation.ipynb

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from helpers.collection_of_experiments import CollectionOfExperiments
from helpers.plot_helpers import *
from helpers.computational_tools import (
    remesh,
    gaussian_remesh,
    select_LatLon,
    Lk_error,
    x_coord,
    y_coord,
)
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmocean
import os

%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings("ignore")
import hvplot.xarray
import holoviews as hv
import matplotlib as mpl
import cartopy.crs as ccrs

from matplotlib.animation import FuncAnimation, PillowWriter

hvplot.output(widget_location="bottom")

In [ ]:
path = "gs://leap-persistent/jbusecke/OM4_m2lines/daily_combined.zarr"
ds = xr.open_dataset(path, engine="zarr", chunks={})
ds

In [ ]:
ZBu = ds["ssu"].sel(experiment="ZB2020")
ZBv = ds["ssv"].sel(experiment="ZB2020")
ZBT = ds["tos"].sel(experiment="ZB2020")

In [ ]:
loresu = ds["ssu"].sel(experiment="unparameterized")
loresv = ds["ssv"].sel(experiment="unparameterized")
loresT = ds["tos"].sel(experiment="unparameterized")

In [ ]:
def KE(u, v, T):
    return 0.5 * (remesh(u**2, T) + remesh(v**2, T))

In [ ]:
ZBke = KE(ZBu, ZBv, ZBT)
loreske = KE(loresu, loresv, ZBT)

In [ ]:
plt.figure(figsize=(15, 15))
np.log10(loreske.isel(time=0)).plot(cmap="inferno", vmin=-3, vmax=0)
plt.axvline(x=-80)
plt.axvline(x=50)
plt.axhline(y=-70)
plt.axhline(y=-10);

# Global plotter

In [ ]:
def update_frame(i, plot_func, fig, ax1, ax2):
    # Clear the current figure to prepare for the next frame's plot.
    plt.clf()

    plot_func(i, fig, ax1, ax2)


def create_gif(plot_func, idx, filename="my-animation.gif", dpi=200, FPS=18, loop=0):
    fig, (ax1, ax2) = plt.subplots(
        2, 1, figsize=(10, 20), subplot_kw={"projection": ccrs.Orthographic()}
    )
    fig.subplots_adjust(hspace=0.05)  # Adjust the spacing if necessary
    ani = FuncAnimation(
        fig,
        update_frame,
        frames=idx,
        fargs=(plot_func, fig, ax1, ax2),
        repeat=bool(loop),
        blit=False,
    )

    writer = PillowWriter(fps=FPS, metadata=dict(artist="Me"))
    ani.save(filename, writer=writer)
    plt.close(fig)


def plot_global(i=0, fig=None, ax1=None, ax2=None):
    if fig is None:
        print("fig is none")
        fig, (ax1, ax2) = plt.subplots(
            2, 1, figsize=(10, 20), subplot_kw={"projection": ccrs.Orthographic()}
        )
        fig.subplots_adjust(hspace=0.05)  # Adjust the spacing if necessary

    time_idx = i * 5
    time_str = loreske.time.isel(time=time_idx).item().strftime("%Y-%m-%d")
    ax1.clear()
    ax2.clear()

    ax1.coastlines(resolution="110m")
    ax1.gridlines()
    im = loreske.isel(time=time_idx).plot(
        ax=ax1,
        transform=ccrs.PlateCarree(),
        norm=mpl.colors.LogNorm(vmin=1e-3, vmax=1e0),
        cmap="inferno",
        add_colorbar=False,
    )
    ax1.set_title("$1/4^o$", fontsize=30)

    ax2.coastlines(resolution="110m")
    ax2.gridlines()
    im = ZBke.isel(time=time_idx).plot(
        ax=ax2,
        transform=ccrs.PlateCarree(),
        norm=mpl.colors.LogNorm(vmin=1e-3, vmax=1e0),
        cmap="inferno",
        add_colorbar=False,
    )
    ax2.set_title("$1/4^o$, ZB2020", fontsize=30)
    cb = plt.colorbar(im, pad=0.02, ax=[ax1, ax2], extend="both", aspect=50, shrink=0.9)
    cb.set_label(label="Surface KE, $\mathrm{m}^2/\mathrm{s}^2$", fontsize=30)
    plt.suptitle(time_str)

In [ ]:
plot_global()

Here, we show the animation of the Surface Kinetic Energy in intervals of 5 days shifting the longitude by 5 degrees each step. 


In [ ]:
%%time
create_gif(
    plot_global,
    range(0, 360, 5),
    filename="./global-ZB2020.gif",
    dpi=200,
    FPS=18,
    loop=0,
)

## EXPERIMENTAL CODE

In [ ]:
%%time
# Adjust matplotlib settings only once

plt.rcParams.update(
    {"font.family": "serif", "figure.subplot.hspace": 0.05, "font.size": 30}
)

# Pre-initialize the figure and axes outside the plotting function
fig, (ax1, ax2) = plt.subplots(
    2, 1, figsize=(10, 20), subplot_kw={"projection": ccrs.Orthographic()}
)
fig.subplots_adjust(hspace=0.05)  # Adjust the spacing if necessary


def plot_global(i=0):
    # Clear axes to prepare for the next frame's plot
    ax1.clear()
    ax2.clear()

    # Set projection and plot properties
    ax1.coastlines(resolution="110m")
    ax1.gridlines()
    ax2.coastlines(resolution="110m")
    ax2.gridlines()

    # Calculate time index and format string
    time_idx = i * 5  # Adjust this based on your indexing needs
    time_str = loreske.time.isel(time=time_idx).item().strftime("%Y-%m-%d")

    # Plotting data on ax1 and ax2 with actual code from your example
    im = loreske.isel(time=time_idx).plot(
        ax=ax1,
        transform=ccrs.PlateCarree(),
        norm=mpl.colors.LogNorm(vmin=1e-3, vmax=1e0),
        cmap="inferno",
        add_colorbar=False,
    )
    ax1.set_title("$1/4^o$", fontsize=30)

    im = ZBke.isel(time=time_idx).plot(
        ax=ax2,
        transform=ccrs.PlateCarree(),
        norm=mpl.colors.LogNorm(vmin=1e-3, vmax=1e0),
        cmap="inferno",
        add_colorbar=False,
    )
    ax2.set_title("$1/4^o$, ZB2020", fontsize=30)

    # Adjust the colorbar
    cb = plt.colorbar(im, pad=0.02, ax=[ax1, ax2], extend="both", aspect=50, shrink=0.9)
    cb.set_label(label="Surface KE, $\mathrm{m}^2/\mathrm{s}^2$", fontsize=30)

    # Set the supertitle for the figure
    fig.suptitle(time_str, fontsize=30)


def update_frame(i):
    plot_global(i)


# Create the animation
ani = FuncAnimation(fig, update_frame, frames=range(0, 360, 5), repeat=False)

# Save the animation
ani.save(
    "optimized-global-ZB2020.gif",
    writer=PillowWriter(fps=18, metadata=dict(artist="Me")),
)
plt.close(fig)

# Regional plotter

In [ ]:
default_rcParams({"figure.subplot.hspace": 0.05, "font.size": 30})


def plot_regions(i=0, lat=slice(15, 70), lon=slice(-100, 0), fig=None):
    if fig is None:
        fig = plt.figure(figsize=(7, 13))
    time_idx = i
    time_str = loreske.time.isel(time=time_idx).item().strftime("%Y-%m-%d")

    ax1 = plt.subplot(
        211,
        projection=ccrs.Orthographic(
            central_latitude=(lat.start + lat.stop) / 2,
            central_longitude=(lon.start + lon.stop) / 2,
        ),
    )
    ax1.coastlines(resolution="110m")
    ax1.gridlines()
    im = (
        loreske.isel(time=time_idx)
        .sel(xh=lon, yh=lat)
        .plot(
            ax=ax1,
            transform=ccrs.PlateCarree(),
            norm=mpl.colors.LogNorm(vmin=1e-3, vmax=1e0),
            cmap="inferno",
            add_colorbar=False,
        )
    )
    ax1.set_title("$1/4^o$", fontsize=30)

    ax2 = plt.subplot(
        212,
        projection=ccrs.Orthographic(
            central_latitude=(lat.start + lat.stop) / 2,
            central_longitude=(lon.start + lon.stop) / 2,
        ),
    )
    # ax2.coastlines(resolution='110m')
    ax2.gridlines()
    im = (
        ZBke.isel(time=time_idx)
        .sel(xh=lon, yh=lat)
        .plot(
            ax=ax2,
            transform=ccrs.PlateCarree(),
            norm=mpl.colors.LogNorm(vmin=1e-3, vmax=1e0),
            cmap="inferno",
            add_colorbar=False,
        )
    )
    ax2.set_title("$1/4^o$, ZB2020", fontsize=30)
    cb = plt.colorbar(im, pad=0.02, ax=[ax1, ax2], extend="both", aspect=50, shrink=0.9)
    cb.set_label(label="Surface KE, $\mathrm{m}^2/\mathrm{s}^2$", fontsize=30)
    plt.suptitle(time_str)

## Plotting the Surface Kinetic Energy in the North Atlantic region.

In [ ]:
plot_Atlantic = lambda idx, fig: plot_regions(
    i=idx, lat=slice(20, 60), lon=slice(-90, -40), fig=fig
)
plot_Atlantic(1461, None)
# plot_Atlantic(1826)

Saving the corresponding animation

In [ ]:
%%time
create_gif(
    plot_Atlantic,
    range(1461, 1826, 5),
    filename="./global-ZB2020-Atlantic.gif",
    dpi=200,
    FPS=18,
    loop=0,
)

## Plotting the Surface Kinetic Energy in the Pacific region.

In [ ]:
plot_Pacific = lambda idx, fig: plot_regions(
    i=idx, lat=slice(20, 60), lon=slice(-240, -190), fig=fig
)
plot_Pacific(-1, None)

Saving the corresponding animation

In [ ]:
%%time
create_gif(
    plot_Pacific,
    range(1461, 1826, 5),
    filename="./global-ZB2020-Pacific.gif",
    dpi=200,
    FPS=18,
    loop=0,
)

## Plotting the Surface Kinetic Energy in the Aghulas region

In [ ]:
plot_Aghulas = lambda idx, fig: plot_regions(
    idx=idx, lat=slice(-50, -10), lon=slice(0, 50), fig=fig
)
plot_Aghulas(-1, None)

Saving the corresponding animation

In [ ]:
%%time
create_gif(
    plot_Aghulas,
    range(1461, 1826, 5),
    filename="./global-ZB2020-Aghulas.gif",
    dpi=200,
    FPS=18,
    loop=0,
)